In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./auth.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                resp = requests.get('https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:' + input_.loc[a, 'Part_Number'] + ')&size=5000&useCache=true&auth=' + input_.loc[a, 'Auth'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                for data in json_data:
                    make = data['fitment']['Vcdb']['MakeName'].strip()
                    model = data['fitment']['Vcdb']['ModelName'].strip()
                    if len(data['fitment']['Vcdb']['Engines']) == 0:
                        engine = ''
                    else:
                        engine = '' if data['fitment']['Vcdb']['Engines'][0]['Liter'] == 0 else str(data['fitment']['Vcdb']['Engines'][0]['Liter']) + 'L'
                        engine += '' if data['fitment']['Vcdb']['Engines'][0]['BlockType'].strip() == '-' else ' ' + data['fitment']['Vcdb']['Engines'][0]['BlockType'].strip() + str(data['fitment']['Vcdb']['Engines'][0]['Cylinders'])
                    year = int(data['fitment']['Vcdb']['Year'])

                    if make not in dict_vehicle:
                        dict_vehicle[make] = {model: {engine: [year]}}
                    else:
                        if model not in dict_vehicle[make]:
                            dict_vehicle[make][model] = {engine: [year]}
                        else:
                            if engine not in dict_vehicle[make][model]:
                                dict_vehicle[make][model][engine] = [year]
                            else:
                                dict_vehicle[make][model][engine] += [year]

                    dict_vehicle[make][model][engine] = sorted(list(set(dict_vehicle[make][model][engine])))
                     
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        list_year = []
                        for engine in dict_vehicle[make][model]:
                            list_year += dict_vehicle[make][model][engine]
                        list_year = sorted(list(set(list_year)))

                        year_min = min(list_year)
                        year_max = max(list_year)
                        if year_min == year_max:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                        else:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                vehicle_1 = '\n'.join(sorted(list_vehicle))

                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        for engine in dict_vehicle[make][model]:
                            year_min = min(dict_vehicle[make][model][engine])
                            year_max = max(dict_vehicle[make][model][engine])
                            if year_min == year_max:
                                if engine == '':
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                                else:
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
                            else:
                                if engine == '':
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                                else:
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

                vehicle_2 = '\n'.join(sorted(list_vehicle))

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                                         'Vehicle_1': vehicle_1,
                                         'Vehicle_2': vehicle_2}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                                     'Auth': input_.loc[a, 'Auth']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Part_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1401

[ok] - 40-10000
[尝试次数：1] - [剩余数量：1381] - [当前时间：14:36:34]

[ok] - 40-10019
[尝试次数：1] - [剩余数量：1380] - [当前时间：14:36:34]

[ok] - 40-10035
[尝试次数：1] - [剩余数量：1379] - [当前时间：14:36:35]

[ok] - 40-10020
[尝试次数：1] - [剩余数量：1378] - [当前时间：14:36:35]

[ok] - 40-1001
[尝试次数：1] - [剩余数量：1377] - [当前时间：14:36:36]

[ok] - 40-10006
[尝试次数：1] - [剩余数量：1376] - [当前时间：14:36:36]

[ok] - 40-10015
[尝试次数：1] - [剩余数量：1375] - [当前时间：14:36:36]

[ok] - 40-10007
[尝试次数：1] - [剩余数量：1374] - [当前时间：14:36:36]

[ok] - 40-1002
[尝试次数：1] - [剩余数量：1373] - [当前时间：14:36:36]

[ok] - 40-10024
[尝试次数：1] - [剩余数量：1372] - [当前时间：14:36:36]

[ok] - 40-10013
[尝试次数：1] - [剩余数量：1371] - [当前时间：14:36:36]

[ok] - 40-10034
[尝试次数：1] - [剩余数量：1370] - [当前时间：14:36:37]

[ok] - 40-1000
[尝试次数：1] - [剩余数量：1369] - [当前时间：14:36:37]

[ok] - 40-10014
[尝试次数：1] - [剩余数量：1368] - [当前时间：14:36:37]

[ok] - 40-10021
[尝试次数：1] - [剩余数量：1367] - [当前时间：14:36:37]

[ok] - 40-1003
[尝试次数：1] - [剩余数量：1366] - [当前时间：14:36:38]

[ok] - 40-10037
[尝试次数：1] - [剩余数量：1365] - [当前时间：14:36:39]

[ok] - 4

[ok] - 40-197
[尝试次数：1] - [剩余数量：1237] - [当前时间：14:38:56]

[ok] - 40-196
[尝试次数：1] - [剩余数量：1236] - [当前时间：14:38:56]

[ok] - 40-155
[尝试次数：1] - [剩余数量：1235] - [当前时间：14:38:59]

[ok] - 40-2000
[尝试次数：1] - [剩余数量：1234] - [当前时间：14:39:00]

[ok] - 40-2006
[尝试次数：1] - [剩余数量：1233] - [当前时间：14:39:10]

[ok] - 40-2001
[尝试次数：1] - [剩余数量：1232] - [当前时间：14:39:15]

[ok] - 40-2005
[尝试次数：1] - [剩余数量：1231] - [当前时间：14:39:17]

[ok] - 40-160
[尝试次数：1] - [剩余数量：1230] - [当前时间：14:39:18]

[ok] - 40-2004
[尝试次数：2] - [剩余数量：1229] - [当前时间：14:39:18]

[ok] - 40-2007
[尝试次数：1] - [剩余数量：1228] - [当前时间：14:39:18]

[ok] - 40-2008
[尝试次数：1] - [剩余数量：1227] - [当前时间：14:39:23]

[ok] - 40-2011
[尝试次数：1] - [剩余数量：1226] - [当前时间：14:39:25]

[ok] - 40-2012
[尝试次数：1] - [剩余数量：1225] - [当前时间：14:39:27]

[ok] - 40-154
[尝试次数：1] - [剩余数量：1224] - [当前时间：14:39:28]

[ok] - 40-2003
[尝试次数：1] - [剩余数量：1223] - [当前时间：14:39:28]

[ok] - 40-2009
[尝试次数：1] - [剩余数量：1222] - [当前时间：14:39:28]

[ok] - 40-190
[尝试次数：1] - [剩余数量：1221] - [当前时间：14:39:28]

[ok] - 40-1911
[尝试次数：1] - [剩余数量：1220]

[ok] - 40-3017L
[尝试次数：1] - [剩余数量：1092] - [当前时间：14:41:20]

[ok] - 40-3027
[尝试次数：1] - [剩余数量：1091] - [当前时间：14:41:21]

[ok] - 40-3018
[尝试次数：1] - [剩余数量：1090] - [当前时间：14:41:22]

[ok] - 40-3019
[尝试次数：1] - [剩余数量：1089] - [当前时间：14:41:22]

[ok] - 40-3024
[尝试次数：2] - [剩余数量：1088] - [当前时间：14:41:22]

[ok] - 40-3032
[尝试次数：1] - [剩余数量：1087] - [当前时间：14:41:24]

[ok] - 40-3026
[尝试次数：1] - [剩余数量：1086] - [当前时间：14:41:25]

[ok] - 40-3034
[尝试次数：1] - [剩余数量：1085] - [当前时间：14:41:25]

[ok] - 40-3033
[尝试次数：1] - [剩余数量：1084] - [当前时间：14:41:26]

[ok] - 40-3015
[尝试次数：2] - [剩余数量：1083] - [当前时间：14:41:26]

[ok] - 40-3030
[尝试次数：1] - [剩余数量：1082] - [当前时间：14:41:26]

[ok] - 40-299
[尝试次数：1] - [剩余数量：1081] - [当前时间：14:41:27]

[ok] - 40-3029L
[尝试次数：1] - [剩余数量：1080] - [当前时间：14:41:27]

[ok] - 40-3031
[尝试次数：1] - [剩余数量：1079] - [当前时间：14:41:27]

[ok] - 40-288
[尝试次数：1] - [剩余数量：1078] - [当前时间：14:41:29]

[ok] - 40-3035
[尝试次数：1] - [剩余数量：1077] - [当前时间：14:41:29]

[ok] - 40-3037
[尝试次数：1] - [剩余数量：1076] - [当前时间：14:41:29]

[ok] - 40-3048
[尝试次数：1] - [剩余数量

[ok] - 43-1410
[尝试次数：1] - [剩余数量：946] - [当前时间：14:42:43]

[ok] - 43-1409
[尝试次数：2] - [剩余数量：945] - [当前时间：14:42:43]

[ok] - 43-1252
[尝试次数：1] - [剩余数量：944] - [当前时间：14:42:43]

[ok] - 43-1416
[尝试次数：1] - [剩余数量：943] - [当前时间：14:42:43]

[ok] - 43-1414
[尝试次数：1] - [剩余数量：942] - [当前时间：14:42:44]

[ok] - 43-1413
[尝试次数：1] - [剩余数量：941] - [当前时间：14:42:44]

[ok] - 43-1415
[尝试次数：1] - [剩余数量：940] - [当前时间：14:42:44]

[ok] - 43-1417
[尝试次数：1] - [剩余数量：939] - [当前时间：14:42:44]

[ok] - 43-1412
[尝试次数：1] - [剩余数量：938] - [当前时间：14:42:44]

[ok] - 43-1418
[尝试次数：1] - [剩余数量：937] - [当前时间：14:42:45]

[ok] - 43-1429
[尝试次数：1] - [剩余数量：936] - [当前时间：14:42:45]

[ok] - 43-1419
[尝试次数：1] - [剩余数量：935] - [当前时间：14:42:45]

[ok] - 40-265
[尝试次数：2] - [剩余数量：934] - [当前时间：14:42:45]

[ok] - 43-1424
[尝试次数：1] - [剩余数量：933] - [当前时间：14:42:45]

[ok] - 43-1171
[尝试次数：1] - [剩余数量：932] - [当前时间：14:42:45]

[ok] - 43-1420
[尝试次数：1] - [剩余数量：931] - [当前时间：14:42:45]

[ok] - 43-1423
[尝试次数：1] - [剩余数量：930] - [当前时间：14:42:46]

[ok] - 43-1437
[尝试次数：1] - [剩余数量：929] - [当前时间：14:4

[ok] - 43-2040
[尝试次数：1] - [剩余数量：800] - [当前时间：14:43:13]

[ok] - 43-1487
[尝试次数：1] - [剩余数量：799] - [当前时间：14:43:13]

[ok] - 43-2038
[尝试次数：1] - [剩余数量：798] - [当前时间：14:43:13]

[ok] - 43-2045
[尝试次数：1] - [剩余数量：797] - [当前时间：14:43:14]

[ok] - 43-2044
[尝试次数：1] - [剩余数量：796] - [当前时间：14:43:14]

[ok] - 43-2041
[尝试次数：1] - [剩余数量：795] - [当前时间：14:43:14]

[ok] - 43-2046
[尝试次数：1] - [剩余数量：794] - [当前时间：14:43:14]

[ok] - 43-2042
[尝试次数：1] - [剩余数量：793] - [当前时间：14:43:14]

[ok] - 43-2047
[尝试次数：1] - [剩余数量：792] - [当前时间：14:43:14]

[ok] - 43-2052
[尝试次数：1] - [剩余数量：791] - [当前时间：14:43:15]

[ok] - 43-2048
[尝试次数：1] - [剩余数量：790] - [当前时间：14:43:15]

[ok] - 43-2043
[尝试次数：1] - [剩余数量：789] - [当前时间：14:43:15]

[ok] - 43-2055
[尝试次数：1] - [剩余数量：788] - [当前时间：14:43:15]

[ok] - 43-2049
[尝试次数：1] - [剩余数量：787] - [当前时间：14:43:15]

[ok] - 43-2032
[尝试次数：1] - [剩余数量：786] - [当前时间：14:43:15]

[ok] - 43-2061
[尝试次数：1] - [剩余数量：785] - [当前时间：14:43:15]

[ok] - 43-2051
[尝试次数：1] - [剩余数量：784] - [当前时间：14:43:15]

[ok] - 43-2057
[尝试次数：1] - [剩余数量：783] - [当前时间：14:

[ok] - 43-3544
[尝试次数：1] - [剩余数量：653] - [当前时间：14:43:49]

[ok] - 43-3511
[尝试次数：1] - [剩余数量：652] - [当前时间：14:43:50]

[ok] - 43-3547
[尝试次数：1] - [剩余数量：651] - [当前时间：14:43:50]

[ok] - 43-3560
[尝试次数：1] - [剩余数量：650] - [当前时间：14:43:50]

[ok] - 43-3549
[尝试次数：1] - [剩余数量：649] - [当前时间：14:43:51]

[ok] - 43-3587
[尝试次数：1] - [剩余数量：648] - [当前时间：14:43:51]

[ok] - 43-3580
[尝试次数：1] - [剩余数量：647] - [当前时间：14:43:51]

[ok] - 43-3552
[尝试次数：1] - [剩余数量：646] - [当前时间：14:43:51]

[ok] - 43-3533
[尝试次数：1] - [剩余数量：645] - [当前时间：14:43:51]

[ok] - 43-3583
[尝试次数：1] - [剩余数量：644] - [当前时间：14:43:51]

[ok] - 43-3566
[尝试次数：1] - [剩余数量：643] - [当前时间：14:43:51]

[ok] - 43-3568
[尝试次数：1] - [剩余数量：642] - [当前时间：14:43:51]

[ok] - 43-3578
[尝试次数：1] - [剩余数量：641] - [当前时间：14:43:52]

[ok] - 43-4002
[尝试次数：1] - [剩余数量：640] - [当前时间：14:43:52]

[ok] - 43-4008
[尝试次数：1] - [剩余数量：639] - [当前时间：14:43:53]

[ok] - 43-4003
[尝试次数：1] - [剩余数量：638] - [当前时间：14:43:53]

[ok] - 43-3581
[尝试次数：1] - [剩余数量：637] - [当前时间：14:43:54]

[ok] - 43-3590
[尝试次数：1] - [剩余数量：636] - [当前时间：14:

[ok] - 43-4313
[尝试次数：1] - [剩余数量：506] - [当前时间：14:44:16]

[ok] - 43-4318
[尝试次数：1] - [剩余数量：505] - [当前时间：14:44:16]

[ok] - 43-4324
[尝试次数：1] - [剩余数量：504] - [当前时间：14:44:16]

[ok] - 43-4319
[尝试次数：1] - [剩余数量：503] - [当前时间：14:44:16]

[ok] - 43-4325
[尝试次数：1] - [剩余数量：502] - [当前时间：14:44:16]

[ok] - 43-4317
[尝试次数：2] - [剩余数量：501] - [当前时间：14:44:17]

[ok] - 43-4322
[尝试次数：1] - [剩余数量：500] - [当前时间：14:44:17]

[ok] - 43-4316
[尝试次数：2] - [剩余数量：499] - [当前时间：14:44:17]

[ok] - 43-4326
[尝试次数：1] - [剩余数量：498] - [当前时间：14:44:17]

[ok] - 43-4320
[尝试次数：1] - [剩余数量：497] - [当前时间：14:44:17]

[ok] - 43-4321
[尝试次数：1] - [剩余数量：496] - [当前时间：14:44:17]

[ok] - 43-4323
[尝试次数：1] - [剩余数量：495] - [当前时间：14:44:18]

[ok] - 43-4332
[尝试次数：1] - [剩余数量：494] - [当前时间：14:44:18]

[ok] - 43-4327
[尝试次数：1] - [剩余数量：493] - [当前时间：14:44:18]

[ok] - 43-4328
[尝试次数：1] - [剩余数量：492] - [当前时间：14:44:18]

[ok] - 43-4331
[尝试次数：1] - [剩余数量：491] - [当前时间：14:44:18]

[ok] - 43-4329
[尝试次数：1] - [剩余数量：490] - [当前时间：14:44:18]

[ok] - 43-4334
[尝试次数：1] - [剩余数量：489] - [当前时间：14:

[ok] - 43-4519
[尝试次数：1] - [剩余数量：359] - [当前时间：14:44:37]

[ok] - 43-4504
[尝试次数：1] - [剩余数量：358] - [当前时间：14:44:37]

[ok] - 43-4515
[尝试次数：1] - [剩余数量：357] - [当前时间：14:44:37]

[ok] - 43-4522
[尝试次数：1] - [剩余数量：356] - [当前时间：14:44:37]

[ok] - 43-4509
[尝试次数：2] - [剩余数量：355] - [当前时间：14:44:37]

[ok] - 43-4521
[尝试次数：1] - [剩余数量：354] - [当前时间：14:44:37]

[ok] - 43-4516
[尝试次数：1] - [剩余数量：353] - [当前时间：14:44:38]

[ok] - 43-4503
[尝试次数：2] - [剩余数量：352] - [当前时间：14:44:38]

[ok] - 43-4524
[尝试次数：1] - [剩余数量：351] - [当前时间：14:44:38]

[ok] - 43-4518
[尝试次数：1] - [剩余数量：350] - [当前时间：14:44:38]

[ok] - 43-4430
[尝试次数：2] - [剩余数量：349] - [当前时间：14:44:38]

[ok] - 43-4529
[尝试次数：1] - [剩余数量：348] - [当前时间：14:44:39]

[ok] - 43-4526
[尝试次数：1] - [剩余数量：347] - [当前时间：14:44:39]

[ok] - 43-4534
[尝试次数：1] - [剩余数量：346] - [当前时间：14:44:39]

[ok] - 43-4527
[尝试次数：1] - [剩余数量：345] - [当前时间：14:44:39]

[ok] - 43-4528
[尝试次数：1] - [剩余数量：344] - [当前时间：14:44:39]

[ok] - 43-4535
[尝试次数：1] - [剩余数量：343] - [当前时间：14:44:39]

[ok] - 43-4531
[尝试次数：1] - [剩余数量：342] - [当前时间：14:

[ok] - 85-1735
[尝试次数：1] - [剩余数量：212] - [当前时间：14:45:44]

[ok] - 85-154
[尝试次数：1] - [剩余数量：211] - [当前时间：14:45:52]

[ok] - 85-140
[尝试次数：1] - [剩余数量：210] - [当前时间：14:45:52]

[ok] - 85-1836
[尝试次数：1] - [剩余数量：209] - [当前时间：14:45:53]

[ok] - 85-1107
[尝试次数：1] - [剩余数量：208] - [当前时间：14:46:00]

[ok] - 85-175
[尝试次数：1] - [剩余数量：207] - [当前时间：14:46:03]

[ok] - 85-192
[尝试次数：1] - [剩余数量：206] - [当前时间：14:46:03]

[ok] - 85-1835
[尝试次数：1] - [剩余数量：205] - [当前时间：14:46:06]

[ok] - 85-20010
[尝试次数：1] - [剩余数量：204] - [当前时间：14:46:09]

[ok] - 85-2001
[尝试次数：1] - [剩余数量：203] - [当前时间：14:46:10]

[ok] - 85-156
[尝试次数：1] - [剩余数量：202] - [当前时间：14:46:11]

[ok] - 85-20020
[尝试次数：1] - [剩余数量：201] - [当前时间：14:46:12]

[ok] - 85-20034
[尝试次数：2] - [剩余数量：200] - [当前时间：14:46:20]

[ok] - 85-2004
[尝试次数：1] - [剩余数量：199] - [当前时间：14:46:23]

[ok] - 85-20040
[尝试次数：1] - [剩余数量：198] - [当前时间：14:46:23]

[ok] - 85-20037
[尝试次数：1] - [剩余数量：197] - [当前时间：14:46:23]

[ok] - 85-178
[尝试次数：1] - [剩余数量：196] - [当前时间：14:46:25]

[ok] - 85-1089
[尝试次数：1] - [剩余数量：195] - [当前时间：14:4

[ok] - 85-4068
[尝试次数：1] - [剩余数量：65] - [当前时间：14:48:52]

[ok] - 85-388
[尝试次数：2] - [剩余数量：64] - [当前时间：14:48:52]

[ok] - 85-43122
[尝试次数：1] - [剩余数量：63] - [当前时间：14:48:52]

[ok] - 85-4315
[尝试次数：1] - [剩余数量：62] - [当前时间：14:48:53]

[ok] - 85-431
[尝试次数：1] - [剩余数量：61] - [当前时间：14:48:53]

[ok] - 85-4316
[尝试次数：1] - [剩余数量：60] - [当前时间：14:48:54]

[ok] - 85-4317
[尝试次数：1] - [剩余数量：59] - [当前时间：14:48:55]

[ok] - 85-432
[尝试次数：1] - [剩余数量：58] - [当前时间：14:48:55]

[ok] - 85-4334
[尝试次数：1] - [剩余数量：57] - [当前时间：14:48:55]

[ok] - 85-433
[尝试次数：1] - [剩余数量：56] - [当前时间：14:48:55]

[ok] - 85-4341
[尝试次数：1] - [剩余数量：55] - [当前时间：14:48:55]

[ok] - 85-4331
[尝试次数：1] - [剩余数量：54] - [当前时间：14:48:56]

[ok] - 85-4344
[尝试次数：1] - [剩余数量：53] - [当前时间：14:48:57]

[ok] - 85-4348
[尝试次数：1] - [剩余数量：52] - [当前时间：14:48:57]

[ok] - 85-4346
[尝试次数：1] - [剩余数量：51] - [当前时间：14:48:58]

[ok] - 85-437
[尝试次数：1] - [剩余数量：50] - [当前时间：14:48:58]

[ok] - 85-4383
[尝试次数：2] - [剩余数量：49] - [当前时间：14:48:59]

[ok] - 85-438
[尝试次数：1] - [剩余数量：48] - [当前时间：14:48:59]

[ok] - 85-4419
